<a href="https://colab.research.google.com/github/MihirsinhChauhan/Cirq/blob/master/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [4]:
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1hFdyPRbIjwcbzG0MRQMD7pNZQcqzsu-3')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [5]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [6]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file.

In [7]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

100%|██████████| 54288/54288 [00:00<00:00, 14415065.56it/s]


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [8]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['age'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['bmi'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_3, *['children'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_17, *['age', 'sex'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_18, *['index', 'age', 'sex'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_19, *['index', 'age', 'smoker'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_20, *['index', 'age', 'region'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_21, *['index', 'bmi', 'sex'], **{})
chart

We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [19]:
your_name = "tarannum"

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [11]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [20]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
296,19,male,26.86900,0,yes,18579.544440
24,37,male,27.18425,2,no,7072.447995
556,46,male,32.43680,1,no,9501.432144
952,30,female,27.55285,1,no,5160.988563
282,30,male,26.81565,1,no,4830.324267


Let us answer some basic questions about the dataset.


**Q1: How many rows and columns does the dataset have?**

In [33]:
# Get the number of rows and columns of the customized DataFrame
num_rows, num_columns = dataframe.shape

# Print the number of rows and columns
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)


Number of rows: 1271
Number of columns: 6


**Q3: What are the column titles of the input variables?**

In [38]:
# Get the column titles (column names)
input_cols = dataframe.columns

# Print the column titles
print("Column titles:",input_cols )

Column titles: Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges'], dtype='object')


In [39]:
input_cols = ['age', 'sex', 'bmi', 'children', 'smoker', 'charges']

**Q4: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [41]:
# Identify categorical columns
categorical_cols = dataframe.select_dtypes(include=['object', 'category']).columns

# Print the categorical column titles
print("Categorical columns:", categorical_cols)

Categorical columns: Index(['sex', 'smoker'], dtype='object')


**Q5: What are the column titles of output/target variable(s)?**

In [43]:
output_cols = ["charges"]

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ai/aakashns/dataviz-cheatsheet

In [ ]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [44]:
!pip install jovian --upgrade -q

In [45]:
import jovian

In [46]:
jovian.commit()

[jovian] Detected Colab notebook...


[jovian] Error: Please provide the project argument for the first commit e.g. jovian.commit(project='my-project')


## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [47]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [48]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[1.90000000e+01, 1.00000000e+00, 2.68690000e+01, 0.00000000e+00,
         1.00000000e+00, 1.85795444e+04],
        [3.70000000e+01, 1.00000000e+00, 2.71842500e+01, 2.00000000e+00,
         0.00000000e+00, 7.07244799e+03],
        [4.60000000e+01, 1.00000000e+00, 3.24368000e+01, 1.00000000e+00,
         0.00000000e+00, 9.50143214e+03],
        ...,
        [5.20000000e+01, 0.00000000e+00, 4.33590000e+01, 3.00000000e+00,
         0.00000000e+00, 1.30093209e+04],
        [4.80000000e+01, 0.00000000e+00, 2.70921000e+01, 4.00000000e+00,
         0.00000000e+00, 1.25572992e+04],
        [3.90000000e+01, 1.00000000e+00, 2.90272500e+01, 1.00000000e+00,
         1.00000000e+00, 2.56067299e+04]]),
 array([[18579.54444 ],
        [ 7072.447995],
        [ 9501.432144],
        ...,
        [13009.3209  ],
        [12557.299158],
        [25606.729875]]))

**Q6: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [49]:
import torch

# Convert NumPy arrays to PyTorch tensors with data type torch.float32
inputs_tensor = torch.tensor(inputs_array, dtype=torch.float32)
targets_tensor = torch.tensor(targets_array, dtype=torch.float32)

In [50]:
inputs = inputs_tensor
targets = targets_tensor


In [ ]:
inputs = ???
targets = ???

In [52]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [53]:
dataset = TensorDataset(inputs, targets)

**Q7: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [59]:
from torch.utils.data import TensorDataset, random_split

# Calculate the validation size and training size
val_percent = 0.2  # You can adjust this value as needed
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size

# Create a TensorDataset
dataset = TensorDataset(inputs, targets)

# Ensure that train_size and val_size sum up to the total number of samples
if train_size + val_size != len(dataset):
    val_size = len(dataset) - train_size  # Adjust val_size to make the sum equal to the dataset length

# Split the dataset into training and validation sets
train_ds, val_ds = random_split(dataset, [train_size, val_size])


Finally, we can create data loaders for training & validation.

**Q8: Pick a batch size for the data loader.**

In [107]:
batch_size = 128

In [108]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [ ]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

Let's save our work by committing to Jovian.

In [90]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.com/ ):
API KEY:


Abort: ignored

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment).


In [110]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q9: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [111]:
import torch.nn as nn

class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)  # Fill with input_size and output_size

    def forward(self, xb):
        out = self.linear(xb)  # Fill with the linear layer
        return out

    def training_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out, targets)  # Fill with the appropriate loss function
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out, targets)  # Fill with the appropriate loss function
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))


Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [112]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [113]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3082, -0.1388,  0.1855, -0.3569, -0.3697,  0.2582]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.3401], requires_grad=True)]

One final commit before we train the model.

In [114]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.com/ ):
API KEY:


Abort: ignored

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [115]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q10: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [116]:
# Use the evaluate function to get the result
result = evaluate(model, val_loader)

# Print the result (validation loss)
print("Validation Loss:", result['val_loss'])


Validation Loss: 241435552.0



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q11: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [120]:
# Define hyperparameters
epochs = 150 # You can adjust this value based on how many training iterations you want
lr = 0.000001   # You can adjust this value based on the learning rate you want to use

# Call the fit function with the specified hyperparameters
history1 = fit(epochs, lr, model, train_loader, val_loader)


Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Epoch [80], val_loss: nan
Epoch [100], val_loss: nan
Epoch [120], val_loss: nan
Epoch [140], val_loss: nan
Epoch [150], val_loss: nan


In [75]:
# Define hyperparameters for the second training run
epochs = 100  # You can adjust this value based on how many training iterations you want
lr = 0.0005   # You can adjust this value based on the learning rate you want to use

# Call the fit function with the specified hyperparameters for the second run
history2 = fit(epochs, lr, model, train_loader, val_loader)


Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Epoch [80], val_loss: nan
Epoch [100], val_loss: nan


In [76]:
# Define hyperparameters for the third training run
epochs = 75   # You can adjust this value based on how many training iterations you want
lr = 0.0001   # You can adjust this value based on the learning rate you want to use

# Call the fit function with the specified hyperparameters for the third run
history3 = fit(epochs, lr, model, train_loader, val_loader)


Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Epoch [75], val_loss: nan


In [82]:
# Define hyperparameters for the fourth training run
epochs = 80   # You can adjust this value based on how many training iterations you want
lr = 0.0003   # You can adjust this value based on the learning rate you want to use

# Call the fit function with the specified hyperparameters for the fourth run
history4 = fit(epochs, lr, model, train_loader, val_loader)


Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Epoch [80], val_loss: nan


In [86]:
# Define hyperparameters for the fourth training run
epochs = 100  # You can adjust this value based on how many training iterations you want
lr = 0.00000001   # You can adjust this value based on the learning rate you want to use

# Call the fit function with the specified hyperparameters for the fourth run
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Epoch [80], val_loss: nan
Epoch [100], val_loss: nan


In [ ]:
# Access validation loss for each training run
for i, history in enumerate(histories):
    print(f"Training Run {i+1} - Validation Losses:")
    for epoch, result in enumerate(history):
        print(f"Epoch {epoch+1}: {result['val_loss']:.4f}")
    print()


In [124]:
# Initialize a list to store validation losses for each run
val_losses = []

# Access validation loss for each training run
for i, history in enumerate(histories):
    run_losses = []  # Validation losses for this run
    for epoch, result in enumerate(history):
        val_loss = result['val_loss']
        run_losses.append(val_loss)
        print(f"Training Run {i+1}, Epoch {epoch+1}: Validation Loss = {val_loss:.4f}")
    val_losses.append(run_losses)  # Store validation losses for this run

# Now, val_losses is a list of lists containing validation losses for each run and epoch


Training Run 1, Epoch 1: Validation Loss = nan
Training Run 1, Epoch 2: Validation Loss = nan
Training Run 1, Epoch 3: Validation Loss = nan
Training Run 1, Epoch 4: Validation Loss = nan
Training Run 1, Epoch 5: Validation Loss = nan
Training Run 1, Epoch 6: Validation Loss = nan
Training Run 1, Epoch 7: Validation Loss = nan
Training Run 1, Epoch 8: Validation Loss = nan
Training Run 1, Epoch 9: Validation Loss = nan
Training Run 1, Epoch 10: Validation Loss = nan
Training Run 1, Epoch 11: Validation Loss = nan
Training Run 1, Epoch 12: Validation Loss = nan
Training Run 1, Epoch 13: Validation Loss = nan
Training Run 1, Epoch 14: Validation Loss = nan
Training Run 1, Epoch 15: Validation Loss = nan
Training Run 1, Epoch 16: Validation Loss = nan
Training Run 1, Epoch 17: Validation Loss = nan
Training Run 1, Epoch 18: Validation Loss = nan
Training Run 1, Epoch 19: Validation Loss = nan
Training Run 1, Epoch 20: Validation Loss = nan
Training Run 1, Epoch 21: Validation Loss = nan
T

**Q12: What is the final validation loss of your model?**

Let's log the final validation loss to Jovian and commit the notebook

In [125]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Please enter your API key ( from https://jovian.com/ ):
API KEY:


Abort: ignored

In [ ]:
jovian.commit(project=project_name, environment=None)

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q13: Complete the following function definition to make predictions on a single input**

In [127]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)  # Pass inputs through the model
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)


In [128]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([4.2000e+01, 1.0000e+00, 2.3867e+01, 2.0000e+00, 1.0000e+00, 2.4236e+04])
Target: tensor([24235.6914])
Prediction: tensor([nan])


In [129]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([3.2000e+01, 1.0000e+00, 3.6065e+01, 2.0000e+00, 0.0000e+00, 5.3277e+03])
Target: tensor([5327.6670])
Prediction: tensor([nan])


In [130]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([2.8000e+01, 1.0000e+00, 2.6171e+01, 2.0000e+00, 0.0000e+00, 5.0560e+03])
Target: tensor([5056.0073])
Prediction: tensor([nan])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog.

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [ ]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/02-insurance-linear-regression" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ai/aakashns/02-insurance-linear-regression


<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
